In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
from linear_mcmc import *

from redTools import *
from Kcorrect import *
from calibTools import *

In [2]:
table   = np.genfromtxt('corrected_mags_linewidth_all.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc = table['pgc']

print len(pgc)

10736


In [13]:
Clusters = allCluster(table, band='i', slope=None)
Clusters_ = allCluster(table, band='w1', slope=None)

key0 = 'NEST_100001'
myCluster = Clusters[key0]
myCluster_ = Clusters_[key0]
print key0, len(myCluster['mag']), len(myCluster_['mag'])
u = len(myCluster['mag'])
v =  len(myCluster_['mag'])
for i, key in enumerate(Clusters):
    if key!=key0:
        myCluster = Clusters[key]
        myCluster_ = Clusters_[key]
        print key, len(myCluster['mag']),len(myCluster_['mag'])
        u+=len(myCluster['mag'])
        v+=len(myCluster_['mag'])
        
    if 1707532 in myCluster['pgc']:
        print 'Warning: ', key
        
             
print u,v    

NEST_100001 79 75
NEST_200092 27 25
NEST_200037 11 11
NEST_200017 0 22
NEST_200005 35 31
NEST_200006 20 22
NEST_100014 0 17
NEST_200003 7 55
NEST_120002 15 15
NEST_100003 0 23
NEST_100002 38 36
NEST_100005 68 62
NEST_200045 13 12
NEST_100007 39 33
NEST_100006 0 46
NEST_100008 38 37
NEST_200016 29 26
NEST_100030 18 17
NEST_200015 0 19
NEST_100018 14 12
NEST_200012 21 23
472 619


In [4]:
Clusters = allCluster(table, band='r', slope=-8.71)
for key in Clusters:
    print key, Clusters[key]['name'], "%.2f"%Clusters[key]['slope'], "%.2f"%Clusters[key]['zp']

NEST_200092 Pegasus -8.71 12.56
NEST_200037 NGC70 Cluster -8.71 14.02
NEST_200017 Abell 539 0.00 0.00
NEST_200005 NGC410 Cluster -8.71 13.36
NEST_200006 NGC507 Cluster -8.71 13.33
NEST_100014 Antila 0.00 0.00
NEST_200003 Abell 262 -8.71 12.91
NEST_100001 Coma -8.71 14.14
NEST_120002 Virgo W -8.71 11.95
NEST_100003 Centaurus 0.00 0.00
NEST_100002 Virgo -8.71 10.15
NEST_100005 Abell 1367 -8.71 14.09
NEST_200045 NGC80 Cluster -8.71 13.94
NEST_100007 Abell 2151 (Hercules) -8.71 15.32
NEST_100006 Hydra 0.00 0.00
NEST_100008 Ursa Major -8.71 10.66
NEST_200016 Abell 2634/66 -8.71 14.72
NEST_100030 Cancer -8.71 13.54
NEST_200015 Fornax 0.00 0.00
NEST_100018 NGC4065 -8.71 14.44
NEST_200012 Abell 400 -8.71 14.21


In [5]:
band = 'i'
Clusters, slope0, zp0 = TF_iter(table, band = band, n_iter=10, verbose=True)
print slope0, zp0

0 -8.392437693157643 0.006774309125389958
1 -8.346798274062236 0.0014958999065198223
2 -8.33735941326729 0.00030937456743293875
3 -8.335417539539975 6.363946315855307e-05
4 -8.335018381475988 1.3090929867886019e-05
5 -8.334936375833335 2.674134419839257e-06
6 -8.334919757143219 5.489015845843416e-07
7 -8.334916209569085 1.1878224879730601e-07
8 -8.334915383022057 1.2862524803481555e-08
9 -8.334915320553481 4.910714831165614e-09
NEST_200092 Pegasus -8.33 12.40
NEST_200037 NGC70 Cluster -8.33 13.71
NEST_200017 Abell 539 0.00 0.00
NEST_200005 NGC410 Cluster -8.33 13.15
NEST_200006 NGC507 Cluster -8.33 13.06
NEST_100014 Antila 0.00 0.00
NEST_200003 Abell 262 -8.33 12.81
NEST_100001 Coma -8.33 13.95
NEST_120002 Virgo W -8.33 11.74
NEST_100003 Centaurus 0.00 0.00
NEST_100002 Virgo -8.33 10.01
NEST_100005 Abell 1367 -8.33 13.91
NEST_200045 NGC80 Cluster -8.33 13.70
NEST_100007 Abell 2151 (Hercules) -8.33 15.07
NEST_100006 Hydra 0.00 0.00
NEST_100008 Ursa Major -8.33 10.44
NEST_200016 Abell 26

ValueError: too many values to unpack

In [ ]:
dye = {"u":"blue","g":"green","r":"red","i":"orange","z":"maroon","w1":"purple", 'w2': "brown"}
cmap = get_cmap(21)
fig, ax = makeFig(band=band, xLim = (1.5,3.1), yLim = (19.5,6.5))

key0 = 'NEST_100002'
myCluster = Clusters[key0]
zp_virgo = myCluster['zp']
mag = myCluster['mag']-zp_virgo
mag_e = myCluster['mag_e']
logWimx = myCluster['logWimx']
logWimx_e = myCluster['logWimx_e']
pgc_ = myCluster['pgc']

for i, key in enumerate(Clusters):
    if key!=key0:
        myCluster = Clusters[key]
        zp_ = myCluster['zp']
        plotCluster(ax, Clusters, nest=key, color='k', offset=zp_-zp_virgo, plotErrors=True, 
                    alpha=0.15, markersize=0) # cmap(i)
        mag = np.concatenate((mag, myCluster['mag']-zp_))
        mag_e = np.concatenate((mag_e, myCluster['mag_e']))
        logWimx = np.concatenate((logWimx, myCluster['logWimx']))
        logWimx_e = np.concatenate((logWimx_e, myCluster['logWimx_e']))
        pgc_ =  np.concatenate((pgc_, myCluster['pgc']))
    else:
        plotCluster(ax, Clusters, nest=key, color='k', offset=0, plotErrors=True, alpha=0.15, markersize=0)
        

x = np.linspace(0,3.5,50)
y = slope0*(x-2.5)
ax.plot(x,y+zp_virgo, 'k--') 
ax.plot(logWimx, mag+zp_virgo, '.', color=dye[band], alpha=0.3)   
print len(mag), len(logWimx), len(logWimx_e)
print slope0, zp0


## Traditional one way linear fit    
mag_ = mag+zp_virgo
logWimx_ = logWimx
logWimx_e_ = logWimx_e
fit, cov = curve_fit(linfit, mag_, logWimx_-2.5, sigma=np.sqrt(logWimx_e_**2+0.01**2))
m, b = fit
me = np.sqrt(cov[0,0])
be = np.sqrt(cov[1,1])
mbe = cov[1,0]

y = np.linspace(5,20,50)
x = m*y+b+2.5
ax.plot(x,y, 'r--')  
m0 = 1./m
b0 = -b/m
dm0 = np.abs(me/m**2)
db0 = np.sqrt((b0**2 )*(be**2/b**2+me**2/m**2-2*mbe/m/b))

Ylm = ax.get_ylim() ; Xlm = ax.get_xlim()
x0 = 0.97*Xlm[0]+0.03*Xlm[1]
y0 = 0.1*Ylm[0]+0.90*Ylm[1]
ax.text(x0,y0, "Slope = "+"%.2f" % m0+'$\pm$'+"%.2f" % dm0, fontsize=12, color='k')


print m0, b0

In [ ]:
from bokeh.plotting import *
from bokeh.embed import components
from bokeh.models import ColumnDataSource, LabelSet, HoverTool, Range1d, Label, TapTool, OpenURL, CustomJS, CrosshairTool

output_notebook()

In [ ]:
if True:
    
    hover = HoverTool(tooltips=[ 
        ("LogWimx", "@LogWimx"),
        (band, "@"+band),
        ("PGC", "@PGC"),
        ])

    hover.point_policy='snap_to_data'
    hover.line_policy='nearest'#'prev'

    TOOLS = ['pan', 'tap', 'wheel_zoom', 'box_zoom', 'reset', 'save']

    p = figure(tools=TOOLS, toolbar_location="below", plot_width=550, plot_height=450)
    p.title.text_font_size = '14pt'
    p.title.text_color = 'green'
    p.grid.grid_line_color="gainsboro"
    dye = {"u":"blue","g":"green","r":"red","i":"orange","z":"maroon","w1":"purple", 'w2': "brown"}
    rang=dye[band] 

    source = ColumnDataSource({'LogWimx': logWimx_, band: mag_, 'PGC': pgc_})
    render = p.circle('LogWimx', band, source=source, size=5, color=rang, alpha=0.7, hover_color='red', hover_alpha=1, hover_line_color='red',
                        
                        # set visual properties for selected glyphs
                        selection_fill_color=rang,

                        # set visual properties for non-selected glyphs
                        nonselection_fill_alpha=1,
                        nonselection_fill_color=rang,)
                      
    p.line(x, y, line_width=1, color="black", line_dash='dashed')
    p.legend.location = "top_right"
    
    p.x_range = Range1d(1.5,3.1)
    p.y_range = Range1d(19.5,6.5)

    p.xaxis.axis_label = 'Log(W^i_mx)'
    p.yaxis.axis_label = band+'* [mag]'
    p.xaxis.axis_label_text_font_size = "14pt"
    p.yaxis.axis_label_text_font_size = "14pt"
    p.grid.grid_line_color="gainsboro"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    
    
    code = """
        
        var index_selected = source.selected['1d']['indices'][0];
        var win = window.open("http://edd.ifa.hawaii.edu/cf4_photometry/get_sdss_cf4.php?pgc="+source.data['PGC'][index_selected]+"#t01", "EDDesn", "width=1200, height=1000");
        try {win.focus();} catch (e){}

    """

    taptool = p.select(type=TapTool)
    taptool.callback = CustomJS(args=dict(source=source), code=code)



    ttp = """
        <div>
            <div>
                <span style="font-size: 16px; color: blue;">PGC:</span>
                <span style="font-size: 16px; font-weight: bold;">@PGC{int}</span>
            </div>
            <div>
                <span style="font-size: 16px; color: blue;">LogWimx:</span>
                <span style="font-size: 16px; font-weight: bold;">@LogWimx</span>
            </div>  
            <div>
                <span style="font-size: 16px; color: blue;">"""+band+""":</span>
                <span style="font-size: 16px; font-weight: bold;">@"""+band+"""{0.2f}</span>
            </div>          
        </div>
    """


    hover = HoverTool(tooltips=ttp, renderers=[render])

    hover.point_policy='snap_to_data'
    hover.line_policy='nearest'
    #hover.mode='vline'
    p.add_tools(hover)   

    cross = CrosshairTool()
    #cross.dimensions='height'
    cross.line_alpha = 0.3
    cross.line_color = 'green'
    p.add_tools(cross)     

    mytext = Label(x=20, y=290, text="Slope= "+"%.2f" % m0+'+-'+"%.2f" % dm0, text_color='black', text_font_size='12pt', x_units='screen', y_units='screen')
    p.add_layout(mytext)
    mytext = Label(x=20, y=260, text="ZP= "+"%.2f" % b0+'+-'+"%.2f" % db0, text_color='black', text_font_size='12pt', x_units='screen', y_units='screen')
    p.add_layout(mytext)    
    
    show(p)